Import packages and processed run metrics

In [1]:
import re
import pandas as pd
df = pd.read_csv('final_run_metrics.csv')
validation_metric = 'Results/val_acc'

Filter data to runs of interest

In [2]:
filtered_df = df.loc[(df.n_epochs < 6)]

Write function to return summaries of run's metric performance

In [3]:
# get best runs for each group
idx = filtered_df.groupby(['dataset', 'method', 'finetune']) \
    [validation_metric].idxmax()  # return index of max validation metric
filtered_df = df.loc[idx]

## Process metrics and get top runs for each
# are large or small metric values are desirable?
descending_metrics = [name for name in filtered_df.columns if re.match('Results', name)]
ascending_metrics = [
    descending_metrics.pop(descending_metrics.index(name))
    for name in descending_metrics
    if re.search('std', name)
]

# all non-metrics columns are used to identify the experimental run
filtered_runs = filtered_df[[
    name for name in filtered_df.columns
    if name not in descending_metrics + ascending_metrics
]]

# rank the metrics
ranked_descending = filtered_df[descending_metrics].rank(
    method='first',
    ascending=False
)
ranked_ascending = filtered_df[ascending_metrics].rank(
    method='first',
    ascending=True
)

# combine and sort the ranked_metrics
ranked_metrics = pd.concat([ranked_descending, ranked_ascending], axis=1)
ranked_metrics = ranked_metrics[sorted(ranked_metrics.columns)]
filtered_ranks = filtered_runs.join(ranked_metrics)


In [9]:
(ranked_metrics == 1).sum(axis = 1)

In [ ]:
def process_run_metrics(df):
    pass

"""
    # get best runs for each group
    idx = df.groupby(['dataset', 'method', 'finetune']) \
        [validation_metric].idxmax()  # return index of max validation metric
    filtered_df = df.loc[idx]

    ## Process metrics and get top runs for each
    # are large or small metric values are desirable?
    descending_metrics = [name for name in filtered_df.columns if re.match('Results', name)]
    ascending_metrics = [
        descending_metrics.pop(descending_metrics.index(name))
        for name in descending_metrics
        if re.search('std', name)
    ]

    # all non-metrics columns are used to identify the experimental run
    filtered_runs = filtered_df[[
        name for name in filtered_df.columns
        if name not in descending_metrics + ascending_metrics
    ]]

    # rank the metrics
    ranked_descending = filtered_df[descending_metrics].rank(
        method='first',
        ascending=False
    )
    ranked_ascending = filtered_df[ascending_metrics].rank(
        method='first',
        ascending=True
    )

    # combine and sort the ranked_metrics
    ranked_metrics = pd.concat([ranked_descending, ranked_ascending], axis=1)
    ranked_metrics = ranked_metrics[sorted(ranked_metrics.columns)]
    filtered_ranks = filtered_runs.join(ranked_metrics)

    return (
        filtered_ranks,
        filtered_df
    )
"""

